# Ciencia de Datos en Python - Proyecto #1

El proyecto consiste en aplicar los conocimientos aprendidos en clase (y apoyandose de referencias adicionales utiles) para crear modelos predictivos de regresi´on lineal uni-variable sencillos de la forma:



$$
y = β0 + β1 ∗ x
$$

Donde:
* y es la variable dependiente
* x es la variable independiente
* β0 es el intercepto de la recta
* β1 es la pendiente de la recta

#### 1. Usando sclicing con NumPy separar los datos en 2 datasets: entrenamiento(80 %) y validacion y pruebas(20 %)

In [ ]:
import numpy as np

# Carga del archivo de entrenamiento
data = np.load('./proyecto_training_data.npy')

# buscamos obtner el 80% de la longitud del conjunto de datos
train_size = int(len(data) * 0.8)

# Utlizando slicing separamos el 80% de entrenamiento y 20% de validacion
train_data = data[:train_size]
validation_data = data[train_size:]

#print(train_size)
#print(validation_data)
#validation_data

print(f"Conjunto de entrenamiento: {train_data.shape}")
print(f"Conjunto de validación/pruebas: {validation_data.shape}")

#### 3. Analisis exploratorio de datos: Para cada variable en el dataset calcular((usando numpy o pandas):
* media
* valor maximo
* valor mınimo
* rango(peak to peak, no el rango del tensor que por ser vector sabemos que es 1)
* desviacion estandar.

In [ ]:
def calculate_statistics(data):
    statistics = []
    for i in range(data.shape[1]):
        column = data[:, i][~np.isnan(data[:, i])]
        mean = np.mean(column)  # Media
        max_value = np.max(column)  # Valor máximo
        min_value = np.min(column)  # Valor mínimo
        range_value = np.ptp(column)  # Rango (peak to peak)
        std_dev = np.std(column)  # Desviación estándar
        
        # Agregar las estadísticas calculadas a la lista
        statistics.append((mean, max_value, min_value, range_value, std_dev))
    return statistics

# Calcular las estadísticas para los conjuntos de entrenamiento y de validación/pruebas
train_statistics = calculate_statistics(train_data) # data entrenamiento 80%
validation_statistics = calculate_statistics(validation_data) # data validacion 20%

# Imprimir las estadísticas para el conjunto de entrenamiento
print("Estadísticas del conjunto de entrenamiento:")
for i, stats in enumerate(train_statistics):
    print(f"Variable {i+1}: Media={stats[0]}, Máximo={stats[1]}, Mínimo={stats[2]}, Rango={stats[3]}, Desviación estándar={stats[4]}")

# Imprimir las estadísticas para el conjunto de validación/pruebas
print("\nEstadísticas del conjunto de validación/pruebas:")
for i, stats in enumerate(validation_statistics):
    print(f"Variable {i+1}: Media={stats[0]}, Máximo={stats[1]}, Mínimo={stats[2]}, Rango={stats[3]}, Desviación estándar={stats[4]}")

#### 4. Para cada variable en el dataset usar seaborn(funci´on distplot https://seaborn.pydata. org/generated/seaborn.distplot.html) para graficar un histograma de la variable.

In [ ]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

# Ignorar FutureWarnings específicos
warnings.filterwarnings("ignore", category=FutureWarning)

# ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']
variable_names = ['SalePrice', 'OverallQual', '1stFlrSF', 'TotRmsAbvGrd', 'YearBuilt', 'LotFrontage']

sns.set(style="whitegrid")


# Crear un histograma para cada variable
for i, var_name in enumerate(variable_names):
    plt.figure(figsize=(10, 6))  # Tamaño de la figura
    # Filtrar NaNs y utilizar histplot para la visualización
    sns.histplot(train_data[:, i][~np.isnan(train_data[:, i])], bins=30, kde=False, color='blue')
    plt.title(f'Histograma de {var_name}')
    plt.xlabel(var_name)
    plt.ylabel('Frecuencia')
    plt.show()

#### 5. Para cada variable independiente x :
